In [8]:
!pip install accelerate
!pip install --upgrade numpy scipy seaborn
!pip install --upgrade torch torchvision

In [10]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
import torch
from huggingface_hub import login
from bs4 import BeautifulSoup
import re
from pandarallel import pandarallel
import logging


pandarallel.initialize(nb_workers=4)
%matplotlib inline
tqdm.pandas()
logging.basicConfig(filename='extraction.log', level=logging.INFO)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [11]:
from getpass import getpass
from huggingface_hub import login

token = getpass("Введите HF token: ")
login(token=token)

Введите HF token:  ········


In [12]:
class ContactExtractorGPU:
    def __init__(self):
        # Конфиг для 8-битного квантования (чтобы хотя бы в kaggle запустить)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4"
        )

        
        self.processor = AutoProcessor.from_pretrained("google/gemma-3-27b-it")
        self.model = AutoModelForImageTextToText.from_pretrained(
            "google/gemma-3-27b-it",
            quantization_config=bnb_config,
            device_map="auto"
        )
        print(f"Модель загружена на: {next(self.model.parameters()).device}")
        print(f"Используется память: {self.model.get_memory_footprint()/1e9:.1f} GB")
        
        self.prompt_template = """Извлеки ВСЕ контактные данные из текста, включая:
        1. Телефоны (в любом формате: +7, 8, международные)
        2. Email-адреса (учти регистр)
        3. Физические адреса (с индексом, городом, улицей)
        4. Ссылки на соцсети (VK, Telegram, WhatsApp)
        5. Ссылки на карты (Яндекс/Google Maps)
        6. Другие контакты (Skype, ICQ)
        
        **Требования:**
        - Верни ТОЛЬКО валидные данные
        - Нормализуй телефоны в формат +7XXXXXXXXXX
        - Разделяй адреса на компоненты (город → улица → дом)
        - Игнорируй дубликаты
        
        Формат ответа (строго JSON):
        ```json
        {{
            "phones": ["+79528190301", ...],
            "emails": ["contact@example.com", ...],
            "addresses": ["Москва, ул. Пушкина 15", ...],
            "social_links": ["https://t.me/username", ...],
            "map_links": ["https://yandex.ru/maps/org/123", ...],
            "other_contacts": {"skype": "login", "icq": "123456"}
        }}
        ```
        """

    def preprocess_html(self, html):
        """Очистка HTML и извлечение текста"""
        soup = BeautifulSoup(html, 'html.parser')
        for tag in ["script", "style", "meta", "head", "svg"]:
            [t.decompose() for t in soup.find_all(tag)]
        return soup.get_text("\n", strip=True)

    def generate(self, text, max_length=3000):
        """Генерация ответа с ограничением длины"""
        try:
            inputs = self.processor(
                self.prompt_template.format(text=text[:max_length]),
                return_tensors="pt",
                truncation=True
            ).to(device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=500,
                    do_sample=True,
                    temperature=0.3
                )
            return self.processor.decode(outputs[0], skip_special_tokens=True)
        except Exception as e:
            print(f"Ошибка генерации: {str(e)}")
            return "{}"

    def parse_response(self, response):
        """Парсинг ответа модели(json'a) с резервными регулярками"""
        try:
            result = json.loads(response.split("```json")[-1].split("```")[0].strip())
        except:
            result = {
                "phones": re.findall(r'(\+7[\s\-()]*\d{3}[\s\-()]*\d{3}[\s\-()]*\d{2}[\s\-()]*\d{2})', response),
                "emails": re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', response),
                "addresses": re.findall(r'([А-ЯЁ][а-яё]+\s*,\s*[А-ЯЁ][а-яё]+\s*,\s*\d+)', response),
                "social_links": re.findall(r'(https?://(?:t\.me|vk\.com|facebook\.com)/[^\s]+', response),
                "map_links": re.findall(r'(https?://(?:yandex\.ru/maps|google\.com/maps)/[^\s]+', response)
            }
        return result

    def process(self, html):
        """Полный пайплайн обработки"""
        text = self.preprocess_html(html)
        response = self.generate(text)
        return self.parse_response(response)

In [13]:
def process_dataframe(input_csv, output_csv):
    # Загрузка данных
    df = pd.read_csv(input_csv)
    df_with_header = pd.concat([pd.DataFrame([df.columns], columns=df.columns), df]).reset_index(drop=True)
    df_with_header.columns = range(len(df.columns))
    
    # Инициализация экстрактора
    extractor = ContactExtractorGPU()
    # Функция для обработки строки
    def process_row(row):
        try:
            contacts = extractor.process(row[1])
            logging.info(f"Обработана строка {row.name}")
            return pd.Series({
                'extracted_phones': "; ".join(contacts.get('phones', [])),
                'extracted_emails': "; ".join(contacts.get('emails', [])),
                'extracted_addresses': "; ".join(contacts.get('addresses', [])),
                'extracted_social_links': "; ".join(contacts.get('social_links', [])),
                'extracted_map_links': "; ".join(contacts.get('map_links', []))
            })
        except Exception as e:
            print(f"Ошибка в строке {row.name}: {str(e)}")
            return pd.Series({f'extracted_{k}': None for k in ['phones', 'emails', 'addresses', 'social_links', 'map_links']})
    
    # Применение к DataFrame
    print("Начало обработки...")
    result_df = df_with_header.parallel_apply(process_row, axis=1)
    final_df = pd.concat([df_with_header, result_df], axis=1)
    
    # Сохранение
    final_df.to_csv(output_csv, index=False)
    print(f"Результат сохранён в {output_csv}")
    return final_df

In [16]:
process_dataframe("/kaggle/input/mts-data/analyzer_analyzer_urls.csv", "processed_contacts.csv")

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

ValueError: The checkpoint you are trying to load has model type `gemma3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.